In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [41]:
d = pd.read_csv('quikr_car.csv')

In [42]:
backup = d.copy()

In [43]:
d=d[d['year'].str.isnumeric()]

In [44]:
d['year']=d['year'].astype(int)

In [45]:
d.info()

<class 'pandas.core.frame.DataFrame'>
Index: 842 entries, 0 to 891
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        842 non-null    object
 1   company     842 non-null    object
 2   year        842 non-null    int64 
 3   Price       842 non-null    object
 4   kms_driven  840 non-null    object
 5   fuel_type   837 non-null    object
dtypes: int64(1), object(5)
memory usage: 46.0+ KB


In [46]:
d=d[d['Price']!='Ask For Price']

In [47]:
d['Price']

0        80,000
1      4,25,000
3      3,25,000
4      5,75,000
6      1,75,000
         ...   
886    3,00,000
888    2,60,000
889    3,90,000
890    1,80,000
891    1,60,000
Name: Price, Length: 819, dtype: object

In [48]:
d['Price']=d['Price'].str.replace(',','').astype(int)

In [49]:
d['kms_driven']=d['kms_driven'].str.split().str.get(0).str.replace(',','')

In [50]:
d=d[d['kms_driven'].str.isnumeric()]

In [51]:
d['kms_driven']=d['kms_driven'].astype(int)

In [52]:
d['fuel_type'].unique()

array(['Petrol', 'Diesel', nan, 'LPG'], dtype=object)

In [53]:
d=d[~d['fuel_type'].isna()]

In [54]:
d.info()

<class 'pandas.core.frame.DataFrame'>
Index: 816 entries, 0 to 889
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        816 non-null    object
 1   company     816 non-null    object
 2   year        816 non-null    int64 
 3   Price       816 non-null    int64 
 4   kms_driven  816 non-null    int64 
 5   fuel_type   816 non-null    object
dtypes: int64(3), object(3)
memory usage: 44.6+ KB


In [55]:
d['name']=d['name'].str.split().str.slice(0,3).str.join(' ')

In [56]:
d=d.reset_index(drop=True)

In [57]:
d.describe()

,year,Price,kms_driven
count,816.000000,8.160000e+02,816.000000
mean,2012.444853,4.117176e+05,46275.531863
std,4.002992,4.751844e+05,34297.428044
min,1995.000000,3.000000e+04,0.000000
25%,2010.000000,1.750000e+05,27000.000000
50%,2013.000000,2.999990e+05,41000.000000
75%,2015.000000,4.912500e+05,56818.500000
max,2019.000000,8.500003e+06,400000.000000


In [58]:
d=d[d['Price']<=6e6]

In [59]:
d.info()

<class 'pandas.core.frame.DataFrame'>
Index: 815 entries, 0 to 815
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        815 non-null    object
 1   company     815 non-null    object
 2   year        815 non-null    int64 
 3   Price       815 non-null    int64 
 4   kms_driven  815 non-null    int64 
 5   fuel_type   815 non-null    object
dtypes: int64(3), object(3)
memory usage: 44.6+ KB


In [60]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [61]:
X = d.drop('Price',axis=1)
Y = d['Price']

In [62]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=661)

In [63]:
lm = LinearRegression()

In [64]:
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

In [65]:
ohe = OneHotEncoder()

In [66]:
ohe.fit(X[['name','company','fuel_type']])

OneHotEncoder()

In [67]:
ct = make_column_transformer((OneHotEncoder(categories=ohe.categories_),['name','company','fuel_type']),remainder='passthrough')

In [68]:
ct

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehotencoder',
                                 OneHotEncoder(categories=[array(['Audi A3 Cabriolet', 'Audi A4 1.8', 'Audi A4 2.0', 'Audi A6 2.0',
       'Audi A8', 'Audi Q3 2.0', 'Audi Q5 2.0', 'Audi Q7', 'BMW 3 Series',
       'BMW 5 Series', 'BMW 7 Series', 'BMW X1', 'BMW X1 sDrive20d',
       'BMW X1 xDrive20d', 'Chevrolet Beat', 'Chevrolet Beat Diesel',
       'Chevrolet Beat LS', 'Chevrolet B...
       'Volkswagen Vento Konekt', 'Volvo S80 Summum'], dtype=object),
                                                           array(['Audi', 'BMW', 'Chevrolet', 'Datsun', 'Fiat', 'Force', 'Ford',
       'Hindustan', 'Honda', 'Hyundai', 'Jaguar', 'Jeep', 'Land',
       'Mahindra', 'Maruti', 'Mercedes', 'Mini', 'Mitsubishi', 'Nissan',
       'Renault', 'Skoda', 'Tata', 'Toyota', 'Volkswagen', 'Volvo'],
      dtype=object),
                                                           array(['Diesel', 'LPG', 'Petrol'], dtype=object)]),
                                 ['name', 'company', 'fuel_type'])])

In [69]:
pipe = make_pipeline(ct,lm)

In [70]:
pipe.fit(X_train,Y_train)

d:\ML\Car Price Prediction\venv\lib\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Audi A3 Cabriolet', 'Audi A4 1.8', 'Audi A4 2.0', 'Audi A6 2.0',
       'Audi A8', 'Audi Q3 2.0', 'Audi Q5 2.0', 'Audi Q7', 'BMW 3 Series',
       'BMW 5 Series', 'BMW 7 Series', 'BMW X1', 'BMW X1 sDrive20d',
       'BMW X1 xDrive20d', 'Chevrolet Beat', 'Chevrolet Beat...
                                                                            array(['Audi', 'BMW', 'Chevrolet', 'Datsun', 'Fiat', 'Force', 'Ford',
       'Hindustan', 'Honda', 'Hyundai', 'Jaguar', 'Jeep', 'Land',
       'Mahindra', 'Maruti', 'Mercedes', 'Mini', 'Mitsubishi', 'Nissan',
       'Renault', 'Skoda', 'Tata', 'Toyota', 'Volkswagen', 'Volvo'],
      dtype=object),
                                                                            array(['Diesel', 'LPG', 'Petrol'], dtype=object)]),
                                                  ['name', 'company',
                                                   'fuel_type'])])),
                ('linearregression', LinearRegression())])

In [71]:
y_pred = pipe.predict(X_test)

In [72]:
y_pred

array([1418159.92058156,  605421.34442677,  321588.3016656 ,
        562548.17828775,  612897.90031228,  644933.95755077,
        474450.85164808,   76233.40000473,  529892.7635308 ,
        391523.29590443,  231466.6656337 ,  105335.3222896 ,
        388041.33676922,  458097.4347083 ,   29446.05835732,
        390106.42503095,  294549.76584005,  358381.17662171,
       2152118.19126175,  529644.56494556,  288260.24985995,
        392640.95642075,  183054.88049272,   26345.39679223,
       1823004.68422578,  556438.23997494,  182286.38492082,
        162772.81922933,  501736.83555334,  516576.34805798,
        233462.53443602, 1622580.92914858, 1092505.3604092 ,
        362689.43374379,  437631.72497082,  487979.04543186,
        156739.69249043,  237909.4446546 ,  405226.43725332,
       1077221.80838024,  403794.54818626,  296956.82931615,
        162772.81922933,  662077.95715818,  119576.49648002,
        231556.60669461,  459496.43417092,  385988.04081837,
        251396.4020068 ,

In [73]:
r2_score(Y_test,y_pred)

0.7687202450532246

In [74]:
pipe.predict(pd.DataFrame([['Maruti Suzuki Swift','Maruti',2024,0,'Petrol']],columns=['name','company','year','kms_driven','fuel_type']))

array([574555.03925876])

In [75]:
import pickle

In [76]:
pickle.dump(pipe,open('LinearRegressionModel.pkl','wb'))